In [ ]:
#pip install lxml
#pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import en_core_web_sm
nlp = en_core_web_sm.load()
from csv import writer
import pandas as pd
import requests
import spacy
import ast
import re

In [ ]:
def soup_function(url):
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url, 'html.parser')
    return soup

def url_construction(pg_start, pg_end, site_name, query_name):
    const_url=[]
    lead_portion  = 'https://www.law.cornell.edu/search/site/'
    trail_portion = '&f%5B0%5D=bundle%3Asupct_node&retain-filters=1&query='
    end_portion   = '=onlysyllabi'
    for i in range(pg_end):
        pg_start_str = str(pg_start)
        url = lead_portion + site_name + '?page=' + pg_start_str + trail_portion + query_name + end_portion
        const_url.append(url)
        pg_start = pg_start+1
        
    return const_url

In [ ]:
website_url = requests.get('https://www.law.cornell.edu/supct/topiclist.html').text
soup = BeautifulSoup(website_url, 'html.parser')

links = []
for data in soup.find_all('div', class_='oneOfThree'):
    for a in data.find_all('a'):
        if a.get('href') != None:
            links.append('https://www.law.cornell.edu'+ a.get('href')) #for getting link

In [ ]:
list_of_urls =[]
category = []
i=1
for link in links:
    print(i)
    try:
        html_content = soup_function(link)

        next_page_url = html_content.find('li', class_='pager-next')
        next_page = next_page_url.a['href']

        match_next_page = re.match(r".*?\?(.*)&f", next_page)
        next_pg_num = re.match(r"page=(.*)$", match_next_page.group(1))
        pg_start = int(next_pg_num.group(1))
        #print(pg_start, type(pg_start))

        last_page_url = html_content.find('li', class_='pager-last last')
        last_page = last_page_url.a['href']

        match_last_page = re.match(r".*?\?(.*)&f", last_page)
        last_pg_num = re.match(r"page=(.*)$", match_last_page.group(1))
        pg_end = int(last_pg_num.group(1))

        match_query = re.match(r".*\/(.*)\?p", next_page)
        site_name = match_query.group(1)
        query_name = site_name + '&scope'

        category.append(site_name)
        list_of_urls.append(url_construction(pg_start, pg_end, site_name, query_name))
        
        i=i+1
        
    except AttributeError:
        print('Error occusred at: ', link)
        pass

print(len(list_of_urls))

The directory - https://www.law.cornell.edu/supct/topiclist.html is arranged in the form of 3 stage structure. Therefore for we breakdown the scrapping of documents in three stages:
* Extract different topics on the home page. 
* Navigate through each topic and store the document links.
* Finally select only document links with respect to the labels present in our dataset.

#### Stage - 1

In [ ]:
ListOfUrls = {'URL' : list_of_urls, 'Category' : category}
my_df_URL = pd.DataFrame(ListOfUrls)
my_df_URL.to_csv('Thesis - Dataset and Transformations/doc2vec/web scrapping files/home_page_topics.csv', index=False, header=True)

#### Stage - 2

In [ ]:
topic_urls = pd.read_csv('Thesis - Dataset and Transformations/doc2vec/web scrapping files/home_page_topics.csv')
topic_urls.tail()

In [ ]:
def LabelCleaner(text):
    Clean_Label = re.sub("[^a-zA-Z]+", " ", text)
    return Clean_Label

In [ ]:
case_name=[]
topic_name=[]
for idx in range(len(topic_urls)):
    print('Running:', idx)
    x = ast.literal_eval(topic_urls['URL'][idx])
    y = LabelCleaner(topic_urls['Category'][idx])
    for url in x:
        soup_sub_url = soup_function(url)
        h3_list = soup_sub_url.find_all('h3',class_='title')
        for h3 in h3_list:
            if h3.find('a'):
                case_name.append(h3.find('a')['href'])
                topic_name.append(y)
    print('completed:', idx)

In [ ]:
case_url_extracted = {'Case URL' : case_name, 'Topic Name' : topic_name}
my_df = pd.DataFrame(case_url_extracted)
my_df.to_csv('Thesis - Dataset and Transformations/doc2vec/web scrapping files/case_urls_all.csv', index=False, header=True)

#### Stage - 3

In [ ]:
case_url_topics = pd.read_csv('Thesis - Dataset and Transformations/doc2vec/web scrapping files/case_urls_all.csv')
case_url_topics.head()

In [ ]:
ListofLabels = ['immigration', 'habeas', 'insurance', 'health',  'environment or environmental or EPA', 
                'contracts or assumpsit', 'injury', 'evidence', 'administrative', 'apa', 'schools', 
                'attorney and fees', 'civil and rights', 'sentencing', 'tax', 'bankruptcy', 'commercial', 
                'erisa', 'corporations', 'securities', 'property', 'criminal', 'procedure', 'international', 
                'elections', 'communications', 'drug and related', 'drugs', 'indians', 'workers and compensation', 
                'antitrust', 'trademark']

In [ ]:
filtered_URL=[]
for idx in range(len(case_url_topics)):
    if case_url_topics['Topic Name'][idx] in ListofLabels:
        filtered_URL.append(case_url_topics['Case URL'][idx])
        
selected_URL = {'Filtered Case URL' : filtered_URL}
df_url = pd.DataFrame(selected_URL) 
df_url.to_csv('Thesis - Dataset and Transformations/doc2vec/web scrapping files/case_urls_filtered.csv', index=False, header=True)
